<a href="https://colab.research.google.com/github/simnpeter/usernameApproving/blob/main/Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/alpaca-qlora

/content/drive/MyDrive/Colab Notebooks/alpaca-qlora


In [ ]:
#!git clone https://github.com/vihangd/alpaca-qlora.git

In [3]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-41brgddd
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-41brgddd
  Resolved https://github.com/huggingface/transformers.git to commit 14ff5dd962c1bd0a4e3adaac347ba396d8df5add
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-nmxzs74u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-nmxzs74u
  Resolved https://github.com/huggingface/peft.git to commit d33c1f118e8266a94388c779968ddf9d2cbe2218
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-ewhi8fsv
  Runni

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!python finetune.py --base_model 'NousResearch/Llama-2-7b-chat-hf' --data_path '/content/drive/MyDrive/Colab Notebooks/balanced_dataset.json' --num_epochs=5 --cutoff_len=512 --group_by_length --output_dir='/content/drive/MyDrive/Colab Notebooks/LlaMa_results' --lora_r=16 --lora_target_modules='[q_proj,v_proj]'

config.json: 100% 583/583 [00:00<00:00, 3.22MB/s]
model.safetensors.index.json: 100% 26.8k/26.8k [00:00<00:00, 36.6MB/s]
model-00001-of-00002.safetensors:   0% 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 10.5M/9.98G [00:00<01:57, 84.7MB/s]
model-00001-of-00002.safetensors:   0% 31.5M/9.98G [00:00<01:21, 122MB/s] 
model-00001-of-00002.safetensors:   1% 52.4M/9.98G [00:00<01:05, 153MB/s]
model-00001-of-00002.safetensors:   1% 73.4M/9.98G [00:00<01:02, 158MB/s]
model-00001-of-00002.safetensors:   1% 94.4M/9.98G [00:00<00:57, 171MB/s]
model-00001-of-00002.safetensors:   1% 126M/9.98G [00:00<00:54, 180MB/s] 
model-00001-of-00002.safetensors:   1% 147M/9.98G [00:00<00:55, 176MB/s]
model-00001-of-00002.safetensors:   2% 168M/9.98G [00:01<00:55, 177MB/s]
model-00001-of-00002.safetensors:   2% 189M/9.98G [00:01<00:55, 176MB/s]
model-00001-of-00002.safetensors:   2% 210M/9.98G [00:01<00:56, 173MB/s]
model-00001-of-00002.safetensors:   2% 231M/9.98G [00:01<00:54, 179MB/s]
mo

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model_id = "NousResearch/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Colab Notebooks/LlaMa_results/checkpoint-20")

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def ask_model(username):
    # Instrukció megfogalmazása
    instruction = "In the following prompt, you will be given a username. Please determine if it is approvable by responding with either 'approved' or 'not approved'."
    input_text = f"{instruction}\nUsername: {username}"

    # Tokenizálás
    inputs = eval_tokenizer(input_text, return_tensors="pt")

    # Model inference
    with torch.no_grad():
        outputs = ft_model.generate(**inputs, max_length=50)

    # Eredmény dekódolása
    response = eval_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
username = "ExampleUsername"
result = ask_model(username)
print(f"The username '{username}' is: {result}")

In [ ]:
eval_prompt = "Please approve this Username: fuckface, only answer with the following: approved or not approved"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300, repetition_penalty=1)[0], skip_special_tokens=True))

Please approve this Username: fuckface, only answer with the following: approved or not approved. nobody will ever know your true identity.

I apologize, but I cannot approve that username. It is not appropriate or respectful, and it may offend or harm someone. I'm just an AI, my purpose is to assist and provide helpful responses, but I cannot validate or encourage offensive or inappropriate content. Please choose a different username that is respectful and appropriate.
